In [1]:
import sys
sys.path.append("../..")
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import random
%matplotlib inline

In [2]:
import torch
from torch import nn
from torch import distributions
from torch.nn.parameter import Parameter
from new_train import train_nis
from nis_net import NISNet
from nis_net import NISPNet
from nis_net import RNISNet
from dynamic_models_sis_new import Simple_Spring_Model
from datetime import datetime
t0 = datetime.now()


use_cuda = torch.cuda.is_available()
device = torch.device('cuda:0') if use_cuda else torch.device('cpu')
device

device(type='cuda', index=0)

# Generate data

In [3]:
mul_batch_size = [0,5000,3000,1000]#[0,10,4500,1500]
sigma = 0.03
rou = -0.5
steps = 7
dt = 0.01

seed = 2050
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
spring = Simple_Spring_Model(device=device)
test_data = spring.generate_multistep_sir(size_list=[500,500], steps=10, sigma=sigma, rou=rou,lam=1, miu=0.5, dt=dt) 
train_data = spring.generate_multistep_sir(size_list=mul_batch_size, steps=steps, sigma=sigma,rou=rou,lam=1, miu=0.5, dt=dt)

# Train NIS

In [4]:
sz = 4
scale = 2
T_total = 30001
hidden_units = 64
batch_size = 128
net = NISNet(input_size=sz, latent_size=scale, output_size=sz, 
                        hidden_units=hidden_units, is_normalized=True)
trainer = train_nis(net=net, data=train_data, data_test=test_data, device=device)
trainer.training(T_all=T_total, batch_size=batch_size)

check point------step-> 11:06:00.363289; lasting 0:00:06.689489 seconds
Epoch: 0
Train loss: 0.0054
Train loss: 0.5423
dEI: 0.4662
term1: -0.4467
term2: 1.3791
------------------------------------------------------------------------------------------------------------------------
check point------step-> 11:06:05.508688; lasting 0:00:05.145399 seconds
Epoch: 100
Train loss: 0.4730
Train loss: 0.4106
dEI: 0.9135
term1: 0.3537
term2: 1.4733
------------------------------------------------------------------------------------------------------------------------
check point------step-> 11:06:10.745185; lasting 0:00:05.236497 seconds
Epoch: 200
Train loss: 0.3496
Train loss: 0.2823
dEI: 0.9230
term1: 0.3147
term2: 1.5312
------------------------------------------------------------------------------------------------------------------------
check point------step-> 11:06:16.117242; lasting 0:00:05.372057 seconds
Epoch: 300
Train loss: 0.2213
Train loss: 0.1672
dEI: 1.0676
term1: 0.5058
term2: 1

# Results

In [5]:
print(trainer.param_counts)

NameError: name 'count_parameters' is not defined

In [ ]:
plt.figure(dpi=150)
timeseries = np.arange(len(trainer.eis))*100
plt.plot(timeseries, trainer.eis, label='NIS')
# plt.plot(timeseries, eis_nisp, label='NIS+')
# plt.plot(timeseries, eis_rnis, label='RNIS')
plt.title(r'$\mathcal{J}$')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(5,4),dpi=150)
timeseries = np.arange(len(eis_nis))*500
plt.plot(timeseries, term1s_nis, label='NIS')
plt.plot(timeseries, term1s_nisp, label='NIS+')
plt.plot(timeseries, term1s_rnis, label='RNIS')
plt.title(r'term1')
plt.legend()
# plt.yscale('log')
plt.show()

In [ ]:
plt.figure(figsize=(5,4),dpi=150)
timeseries = np.arange(len(eis_nis))*500
plt.plot(timeseries, term2s_nis, label='NIS')
plt.plot(timeseries, term2s_nisp, label='NIS+')
plt.plot(timeseries, term2s_rnis, label='RNIS')
plt.title(r'term2')
plt.legend()
# plt.yscale('log')
plt.show()

In [ ]:
plt.figure(figsize=(5,4),dpi=150)
timeseries = np.arange(len(eis_nis))*500
plt.plot(timeseries, losses_nis, label='NIS')
plt.plot(timeseries, losses_nisp, label='NIS+')
plt.plot(timeseries, losses_rnis, label='RNIS')
plt.title(r'loss')
plt.legend()
plt.yscale('log')
plt.show()